# Converting matlab data into python

Use this Jupyter notebook to convert Expo data from .mat to .npy files

In [1]:
import numpy as np
import math, os
import sys
sys.path.insert(0, 'functions/'); # add this path for makeStimulus
import makeStimulus
import helper_fcns as hf
import autoreload

import pdb

# constants - directories
base_loc = '/arc/2.2/p1/plevy/SF_diversity/sfDiv-OriModel/sfDiv-python/';

loc_matData = 'LGN/structures/'; # where are the .mat files?
loc_pyData = 'LGN/structures/'; # where do you want the .npy files?

recArea = 'LGN'

## The original conversion

### Converting files

First, get the .mat files to convert; then, convert unless already done 

In [2]:
loc_matData = 'LGN/recordings/'; # where are the .mat files?

In [53]:
files = os.listdir(loc_matData)

In [54]:
sorted(files)

['m675r01#1[sf11].exxd',
 'm675r01#1[sf11].xml',
 'm675r01#2[sf13].exxd',
 'm675r01#2[sf13].xml',
 'm675r02#1[sf13].exxd',
 'm675r02#1[sf13].xml',
 'm675r03#1[sf13].exxd',
 'm675r03#1[sf13].xml',
 'm675r04#1[sf13].exxd',
 'm675r04#1[sf13].xml',
 'm675r04#4[sfMixLGN].exxd',
 'm675r04#4[sfMixLGN].ofi',
 'm675r04#4[sfMixLGN].waveforms',
 'm675r04#4[sfMixLGN].xml',
 'm675r05#1[sf13].exxd',
 'm675r05#1[sf13].xml',
 'm675r06#1[sf11].exxd',
 'm675r06#1[sf11].xml',
 'm675r06#6[sfMixLGN].exxd',
 'm675r06#6[sfMixLGN].ofi',
 'm675r06#6[sfMixLGN].waveforms',
 'm675r06#6[sfMixLGN].xml',
 'm675r07#1[sf13].exxd',
 'm675r07#1[sf13].xml',
 'm675r08#11[sf13].exxd',
 'm675r08#11[sf13].xml',
 'm675r08#1[sf11].exxd',
 'm675r08#1[sf11].xml',
 'm675r08#5[sfMixLGN].exxd',
 'm675r08#5[sfMixLGN].ofi',
 'm675r08#5[sfMixLGN].waveforms',
 'm675r08#5[sfMixLGN].xml',
 'm675r1#1[sf11].ofi',
 'm675r1#1[sf11].waveforms',
 'm675r1#2[sf13].ofi',
 'm675r1#2[sf13].waveforms',
 'm675r10#1[sf13].exxd',
 'm675r10#1[sf13].ofi'

In [55]:
for i in files:
#     if i.find('#') >= 0:
#         os.rename(loc_matData + i, loc_matData + i.replace('#', ''))
#         print('IGNORE: renaming %s to %s' % (loc_matData + i, loc_matData + i.replace('#', '')))
    if i.find('m675') >=0 and i.find('x') >= 0:
        r_ind = i.find('r');
        rEnd_ind = i.find('#')
        substr_to_replace = i[r_ind+1:rEnd_ind]
#         print('substr: %s' % substr_to_replace)
        new_str = i[0:r_ind+1] + '%02d' % int(substr_to_replace) + i[rEnd_ind:]
        if new_str == i:
            continue;
        os.rename(loc_matData + i, loc_matData + new_str)
        print('renaming %s to %s' % (loc_matData + i, loc_matData + new_str))

In [22]:
files = os.listdir(loc_matData)

In [26]:
files = sorted(files)

In [27]:
# convert individual files
unitName = [];
expType  = [];
unitArea = [];
for i in files:
    # if file has 'sfm' in it and starts with m then 
    if i.find('sfm') >= 0 and i.startswith('m'):
        
        # don't convert if it already exists
        if os.path.exists(loc_pyData + i.replace('.mat', '.npy')):
            if i.endswith('.npy') and i.find('fullWave') == -1: # only add once (not also with .mat)
                unitName.append(i[0:i.find('_')]) # go up to the '_' character
                _, expName = hf.get_exp_ind(loc_pyData, i[0:i.find('_')])
                expType.append(expName);
                unitArea.append(recArea)
            continue;
                
        print("loading: " + i)
        matData = makeStimulus.loadmat(loc_matData + i);
        S = matData.get('S'); # the actual data structure
        _, expName = hf.get_exp_ind(loc_pyData, i[0:i.find('_')])
        
        print("now saving...")
        saveName = loc_pyData + i.replace('.mat', '.npy');
        np.save(saveName, S)
        
        unitName.append(i[0:i.find('_')]) # go up to the '_' character
        expType.append(expName);
        unitArea.append(recArea);

Create/update data list

In [29]:
if os.path.exists(loc_pyData + 'dataList.npy'):
    dataList = np.load(loc_pyData + 'dataList.npy').item();
    dataList['unitName'] = unitName;
    dataList['unitArea'] = unitArea;
    dataList['expType'] = expType;
    np.save(loc_pyData + 'dataList.npy', dataList);
else: # unitType, isolation, comment must be filled in by hand at later time
    dataList = dict();
    dataList['unitName'] = unitName;
    dataList['unitArea'] = unitArea;
    dataList['expType'] = expType;
    dataList['expType'] = expType;
    dataList['isolation'] = [];
    dataList['comment'] = [];
    np.save(loc_pyData + 'dataList.npy', dataList);

In [30]:
dataList = np.load(loc_pyData + 'dataList.npy').item();

Likely not needed, this section was from converting the previous data set where normalization responses where already computed in Matlab

In [ ]:
# change some things around...
for i in dataList['unitName']:

    print("changing: " + i)
    S = np.load(loc_pyData + i + '_sfm.npy').item(); # the actual data structure
    
    if S.get('sfm').get('mod'):
        if S.get('sfm').get('mod').get('normalization') and S.get('sfm').get('mod').get('normalization_py'):
            
            S['sfm']['mod']['norm_old'] = S['sfm']['mod']['normalization'];
            S['sfm']['mod']['normalization'] = S['sfm']['mod']['normalization_py'];
            S['sfm']['mod'].pop('normalization_py');
            
            print("now saving...")
            saveName = loc_pyData + i + '_sfm.npy';
            np.save(saveName, S)

## General 'update python structs' here

Use this section to update something about each cell in the dataList (change appropriate field(s))

In [ ]:
dL = np.load(loc_pyData + 'dataList.npy').item();

In [ ]:
for i in dL['unitName']:
    S = np.load(loc_pyData + i + '_sfm.npy').item();
    if 'descr_model' in S['sfm']:
        S['sfm'].pop('descr_model');
        np.save(loc_pyData + i + '_sfm.npy', S);
        print('Removed descr_model from ' + i);